In [26]:
!pip install requests
import base64
import requests
import datetime
from urllib.parse import urlencode

In [27]:
client_id= '05ce8da1369f40afa35d5d8a0c88aec7'
client_secret = '07c34a7c3bf348fbad425f2c9a57b81b'

In [34]:
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret
        
    def get_client_credentials(self):
        """
        Returns a base64 encoded string
        """
        client_id = self.client_id
        client_secret = self.client_secret
        if client_secret == None or client_id == None:
            raise Exception("You must set client_id and client_secret")
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
        
    
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        return {
            "Authorization": f"Basic {client_creds_b64}"
        }
    
    def get_token_data(self):
        return {
            "grant_type": "client_credentials"
        }
        
    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        r = requests.post(token_url, data=token_data, headers=token_headers)
        if r.status_code not in range (200, 299):
            raise Exception("Could not autenticate client.")
            return False
        data = r.json()
        now = datetime.datetime.now()
        access_token = data['access_token']
        expires_in = data['expires_in']
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True
    
    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires < now:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            elf.perform_auth()
            return self.get_access_token()
        return token
    
    def get_resource_header(self):
        access_token = self.access_token
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        return headers
    
    def get_resource(self, lookup_id, resource_type='albums', version='v1'):
        endpoint = f"https://api.spotify.com/{version}/{resource_type}/{lookup_id}"
        headers = self.get_resource_header() 
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range (200, 299):
            return {}
        return r.json()
    
    def get_album(self, _id):
        return self.get_resource(_id, resource_type='albums')
        
    def get_artist(self, _id):
        return self.get_resource(_id, resource_type='artists')
    
    def search(self, query, search_type="artist"):
        headers = self.get_resource_header()
        endpoint = "https://api.spotify.com/v1/search"
        data = urlencode({"q": query, "type": search_type.lower()})
        lookup_url = f"{endpoint}?{data}"
        r = requests.get(lookup_url, headers=headers)
        if r.status_code not in range (200, 299):
            return {}
        return r.json()
        
    
    

In [42]:
spotify = SpotifyAPI(client_id, client_secret)
spotify.perform_auth()
spotify.search("Turn off the lights", search_type="track") 

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=Turn+off+the+lights&type=track&offset=0&limit=20',
  'items': [{'album': {'album_type': 'single',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/5Igpc9iLZ3YGtKeYfSrrOE'},
       'href': 'https://api.spotify.com/v1/artists/5Igpc9iLZ3YGtKeYfSrrOE',
       'id': '5Igpc9iLZ3YGtKeYfSrrOE',
       'name': 'Chris Lake',
       'type': 'artist',
       'uri': 'spotify:artist:5Igpc9iLZ3YGtKeYfSrrOE'}],
     'available_markets': ['AD',
      'AE',
      'AG',
      'AL',
      'AM',
      'AO',
      'AR',
      'AT',
      'AU',
      'AZ',
      'BA',
      'BB',
      'BD',
      'BE',
      'BF',
      'BG',
      'BH',
      'BI',
      'BJ',
      'BN',
      'BO',
      'BR',
      'BS',
      'BT',
      'BW',
      'BY',
      'BZ',
      'CA',
      'CD',
      'CG',
      'CH',
      'CI',
      'CL',
      'CM',
      'CO',
      'CR',
      'CV',
      'CW',
      'CY',
      'CZ',
      'D

In [45]:
spotify.get_artist("5Igpc9iLZ3YGtKeYfSrrOE")

{'external_urls': {'spotify': 'https://open.spotify.com/artist/5Igpc9iLZ3YGtKeYfSrrOE'},
 'followers': {'href': None, 'total': 231965},
 'genres': ['bass house',
  'deep groove house',
  'edm',
  'electro house',
  'house',
  'melbourne bounce',
  'pop dance',
  'progressive house'],
 'href': 'https://api.spotify.com/v1/artists/5Igpc9iLZ3YGtKeYfSrrOE',
 'id': '5Igpc9iLZ3YGtKeYfSrrOE',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/ab6761610000e5eb78dc17752a33d1435741eb35',
   'width': 640},
  {'height': 320,
   'url': 'https://i.scdn.co/image/ab6761610000517478dc17752a33d1435741eb35',
   'width': 320},
  {'height': 160,
   'url': 'https://i.scdn.co/image/ab6761610000f17878dc17752a33d1435741eb35',
   'width': 160}],
 'name': 'Chris Lake',
 'popularity': 62,
 'type': 'artist',
 'uri': 'spotify:artist:5Igpc9iLZ3YGtKeYfSrrOE'}